## Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system management
import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplolib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


## Read in Data
총 9개의 파일: 
1 main file for training(with target), 1 main file for testing, 1 example submission file, and other files containing additional information about each loan. 

In [ ]:
print(os.listdir("../input/"))

In [ ]:
# Training data
app_train = pd.read_csv('../input/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

training data는 총 307511개, target을 포함한 122개의 feature가 있습니다. 

In [ ]:
# Testing data features
app_test = pd.read_csv('../input/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

# Exploratory Data Analysis

## Examine the distribution of the target column 

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].astype(int).plot.hist();

이 정보를 통해, 이 문제가 imbalanced class problem임을 볼 수 있습니다. 
상환되지 않은 대출금보다 제때 상환된 대출금이 훨씬 더 많습니다.  
상환하지 않은 사람을 제때 상환한 사람으로 잘못 분류하는 문제가 생길 수 있으므로 
imbalance를 반영하기 위해 class에 따라 가중치를 부여하는 방향으로 학습을 진행할 것입니다. 

## Examine missing values

In [ ]:
# column별로 missing values 계산하는 함수 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
    
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df) 
    
        # Table 생성 
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1) # 행 기준 
    
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0: 'Missing values', 1: '% of Total values'})
    
        # Percentage of missing 기준으로 table 내림차순 정렬 
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total values', ascending=False).round(1) 
    
        # Summary information 출력 
        print("선택된 dataframe은 " + str(df.shape[1]) + " 개의 columns입니다.\n"
             "Missing values는 " + str(mis_val_table_ren_columns.shape[0]) + " 개의 columns에 있습니다.")
    
        return mis_val_table_ren_columns

In [ ]:
missing_values = missing_values_table(app_train)
missing_values.head(20)

학습모델을 구축할 때가 되면 N/A값들을 채워야 합니다. 
향후 작업에서는 imputation없이 N/A값을 처리할 수 있는 XGBoost와 같은 모델을 사용할 것입니다. 
결측값이 높은 열을 삭제할 수도 있지만, 그 열이 모델에 유용한지는 아직 알 수 없으므로 당분간 모든 열을 보관합니다. 

## Column Types

In [ ]:
app_train.dtypes.value_counts()

look into `object` (categorical) columns.

In [ ]:
# object dtype을 가지는 각 열들에 있는 unique한 class의 개수  
app_train.select_dtypes('object').apply(pd.Series.nunique, axis=0)

대부분의 categorical variables는 상대적으로 적은 수의 unique entries를 가집니다. 
이제 categorical variables를 처리해줍니다. 

## Encoding Categorical variables


* Label encoding: 범주형 변수의 각 category에 정수를 할당합니다. 새로운 columns는 생성되지 않습니다. 

![image](https://raw.githubusercontent.com/WillKoehrsen/Machine-Learning-Projects/master/label_encoding.png)

* One-hot encoding: 범주형 변수의 각 category에 대해 새 column을 생성합니다. 해당하는 category에는 1을 주고 그렇지 않은 열에는 0을 줍니다. 

![image](https://raw.githubusercontent.com/WillKoehrsen/Machine-Learning-Projects/master/one_hot_encoding.png)

Label encoding의 문제는 의도와 다르게  category에게 임의의 순서를 지정한다는 것입니다. 

따라서 범주형 변수에 대해 두개의 고유값만 있는 경우(ex. Male/Female) Label encoding이 좋지만 두 개 이상의 범주에서는 One-hot encoding이 safe합니다. 

In [ ]:
# label encoder 객체 생성 
le = LabelEncoder()
le_count = 0

for col in app_train:
    if app_train[col].dtype == 'object':
        # 2개 이하의 unique categories를 가지는 경우
        if len(list(app_train[col].unique())) <= 2:
            le.fit(app_train[col])
            # Training 과 test data에 적용
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            le_count += 1

print('%d개의 columns가 label encoded 되었습니다.' % le_count)

In [ ]:
# one-hot encoding 
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training features shape: ', app_train.shape)
print('Testing features shape: ', app_test.shape)

### Aligning Training and Testing data 
training data와 test data는 동일한 feature를 가져야 합니다. One-hot encoding으로 training data에 열이 추가되었습니다. testing data에는 없고 training data에만 있는 열들을 삭제하기 위해서 dataframe을 align해줍니다. 

첫째로, training data에 있는 target column을 빼줍니다. 

In [ ]:
train_labels = app_train['TARGET']

# training, testing data를 align
app_train, app_test = app_train.align(app_test, join = 'inner', axis=1)

# target 다시 넣어줌
app_train['TARGET'] = train_labels

print('Training features shape: ', app_train.shape)
print('Testing features shape: ', app_test.shape)

## Back to EDA

### 이상치 

`describe`method를 사용하여 이상치를 살펴볼 수 있습니다. 

`DAYS_BIRTH`column의 값들은 음수
(현재 대출 신청서와 관련하여 기록되므로)

In [ ]:
app_train['DAYS_BIRTH'].describe()

이 수치를 연도별로 보기위해 -365로 나눠줍니다. 

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

max값이 대략 (365243/365=)1000년이다,, anomaly?

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

Days Employment가 변칙적인 고객들 중 일부를 취해서 그들이 다른 고객들보다 채무불이행률이 더 높거나 낮은 경향이 있는지 알아보자.

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]

print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))

anomaly가 정상치보다 더 낮은 채무불이행율을 가집니다! 
(= anomaly가 더 채무이행을 잘함)

anomaly 처리: 

숫자가 아닌 비정상적인 값을 기입하고, 그 값이 비정상적인지 여부를 나타내는 새로운 boolean column을 만듭니다. (`np.nan`)

In [ ]:
# anomalous flag column 생성
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# anomalous값을 nan으로 
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram');
plt.xlabel('Days Employment');

testing data에도 똑같이 적용시키기

In [ ]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"]==365243
app_test["DAYS_EMPLOYED"].replace({365243:np.nan}, inplace=True)

### Correlations 
Features와 Target간의 Correlations 살펴보기! 

.corr 을 사용하여 모든 variable과 target간의 Pearson correlation coefficient 계산

correlation coefficient 값의 일반적인 해석은 다음과 같이 할 수 있습니다:

* .00-.19 “very weak”
* .20-.39 “weak”
* .40-.59 “moderate”
* .60-.79 “strong”
* .80-1.0 “very strong”

In [ ]:
correlations = app_train.corr()['TARGET'].sort_values()

print('Positive Correlations:\n', correlations.tail(15))
print('\nNegative Correlations:\n', correlations.head(15))

`DAYS_BIRTH` : most positive correlation인데 
대출 당시로부터 해서 마이너스 일수로, 고객의 연령입니다. 
즉, correlation은 positive이지만 `DAYS_BIRTH`의 값은 실제로는 negative입니다. 

이것은 고객이 나이가 들수록, 채무불이행의 가능성이 줄어든다는 것입니다. 
헷갈리니까 해당 feature의 절대값을 취할 것이고, 그렇게 되면 correlation은 negative가 될 것입니다. 


### Effect of Age on Repayment 

In [ ]:
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

고객이 나이가 들면서 target과 negative한 선형관계가 형성되는데, 이는 고객이 나이가 들수록 대출금을 더 잘 상환한다는 것을 의미합니다. 

In [ ]:
plt.figure(figsize = (10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET']==0, 'DAYS_BIRTH'] / 365, label='target == 0')

# KDE plot of loans that were not repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET']==1, 'DAYS_BIRTH'] / 365, label='target == 1')

# Labeling 
plt.xlabel('Age (years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

### Exterior Sources
